In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import when


In [28]:
spark = pyspark.sql.SparkSession.builder.master(
    'local[2]').appName('H1B-3').getOrCreate()
training_data = spark.read.csv(
    '../DATA/training_downsampling_2labels.csv', header=True, inferSchema=True)
test_data = spark.read.csv(
    '../DATA/test_downsampling_2labels.csv', header=True, inferSchema=True)
training_data = training_data.drop(col('_c0'))
test_data = test_data.drop(col('_c0'))
cols = training_data.columns
cols.remove('EMPLOYER_NAME')
training_data = training_data.drop(col('EMPLOYER_NAME'))
training_data = training_data.withColumn(
    "CASE_DURATION", col("CASE_DURATION").cast("int"))
training_data = training_data.withColumn(
    "FULL_TIME_POSITION", col("FULL_TIME_POSITION").cast("int"))
test_data = test_data.drop(col('EMPLOYER_NAME'))
test_data = test_data.withColumn(
    "CASE_DURATION", col("CASE_DURATION").cast("int"))
test_data = test_data.withColumn(
    "FULL_TIME_POSITION", col("FULL_TIME_POSITION").cast("int"))


In [29]:
from pyspark.ml.feature import VectorAssembler
try:
  cols.remove('CASE_STATUS')
except:
  pass
assembler = VectorAssembler(
    inputCols=cols, outputCol="features", handleInvalid='skip')


In [30]:
training_data = assembler.transform(training_data)
test_data = assembler.transform(test_data)
training_data = training_data.withColumn("CASE_STATUS",
                                         when(training_data.CASE_STATUS ==
                                              'CERTIFIED', 1)
                                         .otherwise(0))
test_data = test_data.withColumn("CASE_STATUS",
                                 when(test_data.CASE_STATUS ==
                                      'CERTIFIED', 1)
                                 .otherwise(0))


In [38]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(labelCol='CASE_STATUS',
                 featuresCol='features', maxIter=100, regParam=0.01)

model = lsvc.fit(training_data)


In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predict_train = model.transform(training_data)
predict_test = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol='CASE_STATUS', probabilityCol= 'probability', metricName='f1')



In [40]:
evaluator.evaluate(predict_test)


0.8594384120304902

In [41]:
evaluator.evaluate(predict_train)


0.787482081520442

In [42]:
test_prediction = np.array(predict_test.select('prediction').collect())
train_prediction = np.array(predict_train.select('prediction').collect())
test_label = np.array(predict_test.select('CASE_STATUS').collect())
train_label = np.array(predict_train.select('CASE_STATUS').collect())


In [43]:
from sklearn.metrics import f1_score
f1_score(train_label, train_prediction, average=None), f1_score(
    test_label, test_prediction, average=None)


(array([0.77274784, 0.80221618]), array([0.5182362 , 0.90527507]))

In [44]:
model.save('../saved_models/svm_2labels.sav')

Py4JJavaError: An error occurred while calling o1422.save.
: java.io.IOException: Path ../saved_models/svm_2labels.sav already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallComman